In [ ]:
# =========================================================
# GAME RECOMMENDER SYSTEM
# KNN-based | Interactive | Evaluated
# =========================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import ipywidgets as widgets
from IPython.display import display

# ---------------- DATASET ----------------
data = {
    "game": [
        "Apex Strike", "Shadow Ops", "Dragon Realm", "Kingdom Tactician",
        "Velocity Rush", "Goal Masters", "Nightmare Escape", "City Builder Pro",
        "Cyber Arena", "Mythic Quest", "Warfront Strategy", "Drift Legends",
        "Ultimate Sports 24", "Haunted Asylum", "Life Simulator X"
    ],
    "genres": [
        ["Action", "Shooter"],
        ["Action", "Shooter"],
        ["Adventure", "RPG"],
        ["Strategy", "RPG"],
        ["Racing"],
        ["Sports"],
        ["Horror", "Adventure"],
        ["Simulation", "Strategy"],
        ["Action", "Multiplayer"],
        ["Adventure", "RPG"],
        ["Strategy"],
        ["Racing"],
        ["Sports", "Simulation"],
        ["Horror"],
        ["Simulation"]
    ],
    "description": [
        "Fast paced multiplayer shooter with tactical combat",
        "Stealth based shooting missions with intense action",
        "Fantasy role playing adventure with dragons and magic",
        "Turn based strategy game with kingdom management",
        "High speed racing with realistic physics",
        "Competitive football sports game with career mode",
        "Survival horror game set in a dark abandoned hospital",
        "City building simulation with resource management",
        "Online action arena with fast reflex combat",
        "Epic story driven RPG with character progression",
        "Large scale warfare strategy with troop management",
        "Street racing game with drifting mechanics",
        "Realistic sports simulation with tournaments",
        "Psychological horror exploration experience",
        "Life simulation game focusing on daily activities"
    ],
    "primary_genre": [
        "Shooter", "Shooter", "RPG", "Strategy",
        "Racing", "Sports", "Horror", "Simulation",
        "Action", "RPG", "Strategy", "Racing",
        "Sports", "Horror", "Simulation"
    ]
}

df = pd.DataFrame(data)

# ---------------- FEATURE ENGINEERING ----------------
mlb = MultiLabelBinarizer()
genre_df = pd.DataFrame(mlb.fit_transform(df["genres"]), columns=mlb.classes_)

tfidf = TfidfVectorizer(stop_words="english")
desc_df = pd.DataFrame(
    tfidf.fit_transform(df["description"]).toarray(),
    columns=tfidf.get_feature_names_out()
)

X = pd.concat([genre_df, desc_df], axis=1)

# ---------------- MODEL ----------------
knn = NearestNeighbors(n_neighbors=3, metric="cosine")
knn.fit(X)

# ---------------- RECOMMENDATION FUNCTION ----------------
def recommend_games(selected_genre, user_description=""):
    genre_vector = [1 if g == selected_genre else 0 for g in genre_df.columns]
    desc_vector = (
        tfidf.transform([user_description]).toarray()[0]
        if user_description else np.zeros(desc_df.shape[1])
    )

    query = pd.DataFrame([genre_vector + list(desc_vector)], columns=X.columns)
    distances, indices = knn.kneighbors(query)

    games = df.iloc[indices[0]]["game"].values
    scores = 1 - distances[0]
    return games, scores

# ---------------- MODEL EVALUATION ----------------
def evaluate_model():
    true_labels, predicted_labels = [], []

    for i in range(len(X)):
        query = X.iloc[[i]]
        _, indices = knn.kneighbors(query)
        predicted_labels.append(df.iloc[indices[0][1]]["primary_genre"])
        true_labels.append(df.iloc[i]["primary_genre"])

    labels = sorted(df["primary_genre"].unique())
    cm = confusion_matrix(true_labels, predicted_labels, labels=labels)

    return {
        "accuracy": accuracy_score(true_labels, predicted_labels),
        "precision": precision_score(true_labels, predicted_labels, average="weighted", zero_division=0),
        "recall": recall_score(true_labels, predicted_labels, average="weighted", zero_division=0),
        "f1": f1_score(true_labels, predicted_labels, average="weighted", zero_division=0),
        "confusion_matrix": cm,
        "labels": labels
    }

# ---------------- USER INTERFACE ----------------
genre_dropdown = widgets.Dropdown(
    options=genre_df.columns.tolist(),
    description="Preferred Genre:"
)

description_box = widgets.Textarea(
    description="Game Description:",
    placeholder="Optional (e.g. fast-paced multiplayer action)"
)

button = widgets.Button(description="Generate Results", button_style="success")
output = widgets.Output()

def on_click(b):
    output.clear_output(wait=True)
    with output:
        print("=== RECOMMENDATION RESULTS ===\n")

        games, scores = recommend_games(
            genre_dropdown.value,
            description_box.value
        )

        for g, s in zip(games, scores):
            print(f"{g} → Match Score: {s:.2f}")

        print("\nHigher scores indicate better matches.\n")

        plt.figure(figsize=(6, 4))
        plt.bar(games, scores)
        plt.title("Game Recommendation Match Scores")
        plt.ylabel("Similarity Score")
        plt.tight_layout()
        plt.show()

        print("\n=== MODEL EVALUATION ===\n")

        metrics = evaluate_model()
        print(f"Accuracy : {metrics['accuracy']:.2f}")
        print(f"Precision: {metrics['precision']:.2f}")
        print(f"Recall   : {metrics['recall']:.2f}")
        print(f"F1-Score : {metrics['f1']:.2f}")

        plt.figure(figsize=(5, 4))
        plt.imshow(metrics["confusion_matrix"])
        plt.title("Confusion Matrix")
        plt.colorbar()
        plt.xticks(range(len(metrics["labels"])), metrics["labels"], rotation=45)
        plt.yticks(range(len(metrics["labels"])), metrics["labels"])
        plt.xlabel("Predicted Genre")
        plt.ylabel("Actual Genre")
        plt.tight_layout()
        plt.show()

button.on_click(on_click)

display(genre_dropdown, description_box, button, output)


Dropdown(description='Preferred Genre:', options=('Action', 'Adventure', 'Horror', 'Multiplayer', 'RPG', 'Raci…

Textarea(value='', description='Game Description:', placeholder='Optional (e.g. fast-paced multiplayer action)…

Button(button_style='success', description='Generate Results', style=ButtonStyle())

Output()